---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [32]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [33]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [34]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    state_names = states.values()
    data = []
    
    with open('university_towns.txt') as file:
        line = file.readline()
        
        while line:
            line = line.replace('\n', '')
            line = line.split('(')[0]
#             line = line.split('[')[0]
            line = line.strip()
            data.append(line)  
            
            line = file.readline()

    
    category = {}
    while len(data) > 0:
    
        for i in range(len(data)-1, -1, -1):
            if '[' in data[i]:
                category[data[i]] = data[i+1:]
                break
        data = data[0:i]
    
        
    rows = []
    for key, value in category.items():
        for city in value:
            a_row = [key.split('[')[0], city]
            rows.append(a_row)
            
    answer = pd.DataFrame(rows, columns = ["State", "RegionName"]) 
    
    return answer

get_list_of_university_towns()

# get_list_of_university_towns().groupby('State').size()

,State,RegionName
0,Wyoming,Laramie
1,Wisconsin,Appleton
2,Wisconsin,Eau Claire
3,Wisconsin,Green Bay
4,Wisconsin,La Crosse
...,...,...
512,Alabama,Livingston
513,Alabama,Montevallo
514,Alabama,Troy
515,Alabama,Tuscaloosa


In [35]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdp = pd.read_excel('gdplev.xls')
    gdp = gdp.loc[:, ['Unnamed: 4', 'Unnamed: 6'] ]
    gdp.rename(columns = {'Unnamed: 4': 'YearQuarter', 'Unnamed: 6': 'GDP in billions of chained 2009 dollars'}, 
               inplace = True)
    
    gdp = gdp[gdp['YearQuarter'].str[0] == '2']

    for i in range(220, 284):
        a_gdp = gdp.loc[i, 'GDP in billions of chained 2009 dollars']
        previous_gdp = gdp.loc[i-1, 'GDP in billions of chained 2009 dollars']
        next_gdp = gdp.loc[i+1, 'GDP in billions of chained 2009 dollars']
        
        if a_gdp < previous_gdp and next_gdp < a_gdp:
            
            return gdp.loc[i,'YearQuarter']
    


get_recession_start()

'2008q3'

In [36]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    gdp = pd.read_excel('gdplev.xls')
    gdp = gdp.loc[:, ['Unnamed: 4', 'Unnamed: 6'] ]
    gdp.rename(columns = {'Unnamed: 4': 'YearQuarter', 'Unnamed: 6': 'GDP in billions of chained 2009 dollars'}, 
               inplace = True)
    
    gdp = gdp[gdp['YearQuarter'].str[0] == '2']
    
    recess_st = get_recession_start()
    
    recess_st_idx = gdp.index[gdp['YearQuarter'] == recess_st].values[0]
    


    for i in range(recess_st_idx+1, 285):
        a_gdp = gdp.loc[i, 'GDP in billions of chained 2009 dollars']
        previous_gdp = gdp.loc[i-1, 'GDP in billions of chained 2009 dollars']
        pre_previous_gdp = gdp.loc[i-2, 'GDP in billions of chained 2009 dollars']
        
        if a_gdp > previous_gdp and previous_gdp > pre_previous_gdp:
            
            return gdp.loc[i,'YearQuarter']

        
        
get_recession_end()

'2009q4'

In [37]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls')
    gdp = gdp.loc[:, ['Unnamed: 4', 'Unnamed: 6'] ]
    gdp.rename(columns = {'Unnamed: 4': 'YearQuarter', 'Unnamed: 6': 'GDP in billions of chained 2009 dollars'}, 
               inplace = True)
    
    gdp = gdp[gdp['YearQuarter'].str[0] == '2']
    
    recess_st = get_recession_start()
    recess_st_idx = gdp.index[gdp['YearQuarter'] == recess_st].values[0]
    
  
    recess_end = get_recession_end()
    recess_end_idx = gdp.index[gdp['YearQuarter'] == recess_end].values[0]
    

    recess_df = gdp.loc[recess_st_idx: recess_end_idx]
    bottom_gdp = recess_df['GDP in billions of chained 2009 dollars'].min(axis = 0)
    
    bottom_year = recess_df.loc[recess_df['GDP in billions of chained 2009 dollars'] == bottom_gdp, 'YearQuarter']
    
    return bottom_year.values[0]      
    
    

get_recession_bottom()

'2009q2'

In [66]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    dt_names = pd.read_csv('City_Zhvi_AllHomes.csv', usecols = [1,2])
    
    cols = pd.read_csv('City_Zhvi_AllHomes.csv').columns.values

    dt_after_2000 = pd.read_csv('City_Zhvi_AllHomes.csv', usecols = [c for c in cols if c[0] == '2'])
    dt_quartly = dt_after_2000.groupby(pd.PeriodIndex(dt_after_2000.columns, freq ='Q'), axis=1).mean()
    dt_quartly.rename(columns = lambda c: str(c).lower(), inplace = True)
    
    housing_df = pd.merge(dt_names, dt_quartly, how = 'outer', left_index = True, right_index = True)
    
    full_names = []
    for acro in housing_df.loc[:, 'State']:
        full_name = states[acro]
        full_names.append(full_name)
        
    housing_df.insert(0, 'State_full', full_names)
    del housing_df['State']
    housing_df.rename(columns = {'State_full': 'State'}, inplace = True)
  
    
    housing_df.reset_index(inplace = True)
    housing_df.set_index(['State', 'RegionName'], inplace = True)
    del housing_df['index']
    
  
        
    return housing_df

          
convert_housing_data_to_quarters()
# convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q3"]

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [67]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    housing_price = convert_housing_data_to_quarters()
    
    start = get_recession_start()
    start_idx = housing_price.columns.get_loc(start)
    
    bottom = get_recession_bottom()
    bottom_idx = housing_price.columns.get_loc(bottom)
    
    recession_price = housing_price.iloc[:, start_idx:bottom_idx+1]
    recession_price.reset_index(inplace = True)
    
    
    UT = get_list_of_university_towns()

    
    df = pd.merge(UT, recession_price, how = 'outer', on = UT.columns.tolist(), indicator = '_flag')
    UT_house = df.loc[df['_flag'] == 'both'].copy()
    nonUT_house = df.loc[df['_flag'] == 'right_only'].copy()
    
    
    UT_house['diff'] = UT_house['2008q3'] - UT_house['2009q2']
    nonUT_house['diff'] = nonUT_house['2008q3'] - nonUT_house['2009q2']
    
    
    ttest_result = ttest_ind(UT_house['diff'], nonUT_house['diff'], nan_policy = 'omit')
    
    if ttest_result[1] <0.01:
        different = True
    
    p = ttest_result[1]
    
    if ttest_result[0] > 0:
        better = 'non-university town'
    else: 
        better = 'university town'
        
        
    return (different, p, better)



run_ttest()

(True, 0.004325214853459962, 'university town')